# Parsing Data

In [ ]:
#Download connll no alt
#SAX parsing
#https://docs.python.org/2/library/xml.sax.html
#https://wiki.python.org/moin/Sax

In [ ]:
#read conll corpus
#cumpare anotators

In [4]:
import xml.sax
class InkscapeSvgHandler(xml.sax.ContentHandler):
    def __init__(self, teacher_id):
        self.teacher_id=teacher_id
        
    def startDocument(self):
        self.parsed_data=[]
        
    def startElement(self, name, attrs):
        self.tag = name
        if name == "MISTAKE":
            self.current_mistake={}
            for (k,v) in attrs.items():
                self.current_mistake[k]=v
            self.current_mistake['teacher_id']=self.teacher_id
            
    def characters(self, content):
        if self.tag=='TYPE':
            self.current_mistake['type']=content
            self.tag=''
        if self.tag=='CORRECTION':
            self.current_mistake['correction']=content
            self.tag=''
        
    def endElement (self, name):
        if name == "MISTAKE":
            self.parsed_data.append(self.current_mistake)
            
class HubErrorHandler(xml.sax.ErrorHandler):
    def __init__(self):
        pass
    def error(self, exception):
        import sys
        print ("Error, exception: %s\n" % exception)
    def fatalError(self, exception):
        print ("Fatal Error, exception: %s\n" % exception)

parser = xml.sax.make_parser()
content_parser_8= InkscapeSvgHandler(8)
parser.setContentHandler(content_parser_8)
parser.setErrorHandler(HubErrorHandler())
parser.parse(open("conll/noalt/official-2014.0.conll.ann","r"))
content_parser_9= InkscapeSvgHandler(9)
parser.setContentHandler(content_parser_9)
parser.parse(open("conll/noalt/official-2014.1.conll.ann","r"))
results = content_parser_8.parsed_data + content_parser_9.parsed_data

In [5]:
import pandas as pd
df = pd.DataFrame(columns=['teacher_id','type','correction','nid','pid','sid','start_token','end_token'], data=results)

In [6]:
df.to_csv('erors.csv',encoding='utf8', index=False)

# Statistics

In [9]:
df['chars']=df.apply(lambda row: set(range(int(row['start_token'])-1, int(row['end_token']))), axis=1)

In [10]:
df.head()

,teacher_id,type,correction,nid,pid,sid,start_token,end_token,chars
0,8,ArtOrDet,\n,1,1,1,3,4,"{2, 3}"
1,8,Nn,diseases,1,1,2,3,4,"{2, 3}"
2,8,Trans,",",1,1,7,1,1,{0}
3,8,Mec,diagnosed,1,1,8,3,4,"{2, 3}"
4,8,Prep,\n,1,1,8,4,5,"{3, 4}"


In [11]:
df['type'].value_counts()

Wci         819
ArtOrDet    774
Mec         721
Prep        601
Nn          443
Rloc-       293
Vt          283
SVA         259
Pref        249
Vform       223
Wform       180
Trans       171
Others       93
Vm           86
Ssub         78
WOinc        76
V0           68
Pform        65
Um           54
WOadv        39
Npos         34
Srun         28
Spar         27
Wtone        24
Sfrag        14
Smod          5
Wa            1
Name: type, dtype: int64

In [12]:
df[df['teacher_id']==9]['type'].value_counts()

Mec         493
Wci         479
ArtOrDet    443
Prep        390
Nn          228
Rloc-       199
SVA         154
Pref        153
Vt          150
Wform       103
Vform        91
Trans        78
WOinc        54
Others       49
Um           42
V0           37
Vm           37
WOadv        27
Spar         24
Srun         21
Pform        18
Npos         15
Wtone        15
Ssub         10
Smod          5
Sfrag         1
Wa            1
Name: type, dtype: int64

In [14]:
df['correction'].value_counts()[:20]

\n       1046
,         265
a         191
the       138
to         78
are        78
their      71
of         71
in         54
is         51
about      47
have       47
for        45
on         45
they       44
and        38
with       36
lives      34
.          30
be         27
Name: correction, dtype: int64

In [14]:
df[df['teacher_id']==8]['type'].value_counts()

Wci         340
ArtOrDet    331
Mec         228
Nn          215
Prep        211
Vt          133
Vform       132
SVA         105
Pref         96
Rloc-        94
Trans        93
Wform        77
Ssub         68
Vm           49
Pform        47
Others       44
V0           31
WOinc        22
Npos         19
Sfrag        13
WOadv        12
Um           12
Wtone         9
Srun          7
Spar          3
Name: type, dtype: int64

In [15]:
df[df['teacher_id']==9]['type'].value_counts()

Mec         493
Wci         479
ArtOrDet    443
Prep        390
Nn          228
Rloc-       199
SVA         154
Pref        153
Vt          150
Wform       103
Vform        91
Trans        78
WOinc        54
Others       49
Um           42
V0           37
Vm           37
WOadv        27
Spar         24
Srun         21
Pform        18
Npos         15
Wtone        15
Ssub         10
Smod          5
Sfrag         1
Wa            1
Name: type, dtype: int64

as you can see teacher 9 - find much more errors 

# Find inter annotator agreement  = 0.146

In [30]:
# find same errors
grouped_mistakes={}
for key, values in df.groupby(['nid','pid','sid','type','correction','start_token','end_token']):
    grouped_mistakes[key]=values

In [36]:
correct_count =0
wrong_count=0
for group in grouped_mistakes.values():
    len_8 = group[group['teacher_id']==8].shape[0]
    len_9 = group[group['teacher_id']==9].shape[0]
    wrong_count+= max(len_8,len_9) - min(len_8,len_9)
    correct_count+= min(len_8,len_9)  

In [39]:
total = wrong_count + correct_count

In [40]:
inter_annotator_agreement = correct_count/total
inter_annotator_agreement

0.14641494275959027

# Inter annotatorconflicts analysis

In [42]:
grouped_mistakes={}
for key, values in df.groupby(['nid','pid','sid']):
    grouped_mistakes[key]=values

In [19]:
def compare_sets(set1,set2):
    intersection = set1.intersection(set2)
    if len(intersection)==0:
        return True
    return (len(intersection)==len(set1)) and (len(intersection)==len(set2))

def compare_types(type1,type2):
    return type1==type2

In [20]:
def get_error_details(data):
    return {'teacher_id' : data['teacher_id'],'type':data['type'],'correction':data['correction'],'chars':data['chars']}

In [21]:
def find_annotattion_errors(paragraph_errors):
    paragraph_errors_8=paragraph_errors[paragraph_errors['teacher_id']==8]
    paragraph_errors_8
    
    paragraph_errors_9=paragraph_errors[paragraph_errors['teacher_id']==9]
    paragraph_errors_9
    
    diff=[]
    for index_e8, e8 in paragraph_errors_8.iterrows():
        for index_e9, e9 in paragraph_errors_9.iterrows():
            if(compare_types(e8['type'],e9['type'])==False):
                diff.append(('type_error',get_error_details(e8),get_error_details(e9)))
            if(compare_sets(e8['chars'],e9['chars'])==False):
                diff.append(('sets_error',get_error_details(e8),get_error_details(e9)))
            
    return diff

In [23]:
grouped_mistakes={}
for key, values in df.groupby(['nid','pid','sid']):
    grouped_mistakes[key]=find_annotattion_errors(values)
    if len(grouped_mistakes[key])>0:
        print(key)
        print(grouped_mistakes[key])

('1', '1', '1')
[('type_error', {'teacher_id': 8, 'type': 'ArtOrDet', 'correction': '\n', 'chars': {2, 3}}, {'teacher_id': 9, 'type': 'Rloc-', 'correction': '\n', 'chars': {2, 3}})]
('1', '1', '7')
[('type_error', {'teacher_id': 8, 'type': 'Trans', 'correction': ',', 'chars': {0}}, {'teacher_id': 9, 'type': 'Mec', 'correction': ',', 'chars': {0}})]
('1', '1', '8')
[('type_error', {'teacher_id': 8, 'type': 'Mec', 'correction': 'diagnosed', 'chars': {2, 3}}, {'teacher_id': 9, 'type': 'Rloc-', 'correction': 'diagnosed', 'chars': {2, 3, 4}}), ('sets_error', {'teacher_id': 8, 'type': 'Mec', 'correction': 'diagnosed', 'chars': {2, 3}}, {'teacher_id': 9, 'type': 'Rloc-', 'correction': 'diagnosed', 'chars': {2, 3, 4}}), ('type_error', {'teacher_id': 8, 'type': 'Mec', 'correction': 'diagnosed', 'chars': {2, 3}}, {'teacher_id': 9, 'type': 'ArtOrDet', 'correction': 'a', 'chars': {5}}), ('type_error', {'teacher_id': 8, 'type': 'Mec', 'correction': 'diagnosed', 'chars': {2, 3}}, {'teacher_id': 9, '

('15', '2', '0')
[('type_error', {'teacher_id': 8, 'type': 'Ssub', 'correction': 'how', 'chars': {35, 36}}, {'teacher_id': 9, 'type': 'Others', 'correction': 'who are', 'chars': {3}}), ('type_error', {'teacher_id': 8, 'type': 'Ssub', 'correction': 'how', 'chars': {35, 36}}, {'teacher_id': 9, 'type': 'Prep', 'correction': 'to', 'chars': {6, 7}}), ('type_error', {'teacher_id': 8, 'type': 'Ssub', 'correction': 'how', 'chars': {35, 36}}, {'teacher_id': 9, 'type': 'Vform', 'correction': 'reveal', 'chars': {8, 7}}), ('type_error', {'teacher_id': 8, 'type': 'Ssub', 'correction': 'how', 'chars': {35, 36}}, {'teacher_id': 9, 'type': 'Npos', 'correction': "'s", 'chars': {15}}), ('type_error', {'teacher_id': 8, 'type': 'Ssub', 'correction': 'how', 'chars': {35, 36}}, {'teacher_id': 9, 'type': 'Prep', 'correction': '\n', 'chars': {17, 18, 19}}), ('type_error', {'teacher_id': 8, 'type': 'Ssub', 'correction': 'how', 'chars': {35, 36}}, {'teacher_id': 9, 'type': 'Rloc-', 'correction': 'how', 'chars':

('16', '1', '2')
[('type_error', {'teacher_id': 8, 'type': 'Mec', 'correction': ',', 'chars': {32}}, {'teacher_id': 9, 'type': 'Srun', 'correction': '. All', 'chars': {4, 5, 6}}), ('type_error', {'teacher_id': 8, 'type': 'Mec', 'correction': ',', 'chars': {32}}, {'teacher_id': 9, 'type': 'Nn', 'correction': 'offices', 'chars': {24, 25}}), ('type_error', {'teacher_id': 8, 'type': 'Mec', 'correction': ',', 'chars': {32}}, {'teacher_id': 9, 'type': 'Wci', 'correction': 'pedestrians', 'chars': {28, 29}})]
('16', '1', '7')
[('type_error', {'teacher_id': 8, 'type': 'Pref', 'correction': '\n', 'chars': {6, 7}}, {'teacher_id': 9, 'type': 'ArtOrDet', 'correction': '\n', 'chars': {6, 7}}), ('type_error', {'teacher_id': 8, 'type': 'Pref', 'correction': '\n', 'chars': {6, 7}}, {'teacher_id': 9, 'type': 'ArtOrDet', 'correction': 'a', 'chars': {13, 14}}), ('type_error', {'teacher_id': 8, 'type': 'Pref', 'correction': '\n', 'chars': {6, 7}}, {'teacher_id': 9, 'type': 'Mec', 'correction': ',', 'chars'

('24', '5', '4')
[('type_error', {'teacher_id': 8, 'type': 'Vform', 'correction': 'getting', 'chars': {0, 1}}, {'teacher_id': 9, 'type': 'Pref', 'correction': 'he', 'chars': {0}}), ('sets_error', {'teacher_id': 8, 'type': 'Vform', 'correction': 'getting', 'chars': {0, 1}}, {'teacher_id': 9, 'type': 'Pref', 'correction': 'he', 'chars': {0}}), ('type_error', {'teacher_id': 8, 'type': 'Vform', 'correction': 'getting', 'chars': {0, 1}}, {'teacher_id': 9, 'type': 'Wci', 'correction': 'said', 'chars': {14, 15}}), ('type_error', {'teacher_id': 8, 'type': 'Vform', 'correction': 'diseased', 'chars': {20, 21}}, {'teacher_id': 9, 'type': 'Pref', 'correction': 'he', 'chars': {0}}), ('type_error', {'teacher_id': 8, 'type': 'Vform', 'correction': 'diseased', 'chars': {20, 21}}, {'teacher_id': 9, 'type': 'Wci', 'correction': 'said', 'chars': {14, 15}})]
('24', '5', '5')
[('type_error', {'teacher_id': 8, 'type': 'Vform', 'correction': 'knew', 'chars': {5, 6}}, {'teacher_id': 9, 'type': 'Wci', 'correct

('28', '4', '0')
[('type_error', {'teacher_id': 8, 'type': 'Trans', 'correction': 'despite', 'chars': {2, 3}}, {'teacher_id': 9, 'type': 'Vform', 'correction': 'spending', 'chars': {8, 9}}), ('type_error', {'teacher_id': 8, 'type': 'Trans', 'correction': 'despite', 'chars': {2, 3}}, {'teacher_id': 9, 'type': 'ArtOrDet', 'correction': 'the', 'chars': {13}}), ('type_error', {'teacher_id': 8, 'type': 'Trans', 'correction': 'despite', 'chars': {2, 3}}, {'teacher_id': 9, 'type': 'ArtOrDet', 'correction': '\n', 'chars': {16, 17}}), ('type_error', {'teacher_id': 8, 'type': 'Trans', 'correction': 'despite', 'chars': {2, 3}}, {'teacher_id': 9, 'type': 'Wci', 'correction': 'aspects', 'chars': {19, 20}}), ('type_error', {'teacher_id': 8, 'type': 'Others', 'correction': 'nature', 'chars': {5}}, {'teacher_id': 9, 'type': 'Trans', 'correction': 'and', 'chars': {8, 7}}), ('type_error', {'teacher_id': 8, 'type': 'Others', 'correction': 'nature', 'chars': {5}}, {'teacher_id': 9, 'type': 'Vform', 'corre

...

('38', '4', '2')
[('type_error', {'teacher_id': 8, 'type': 'Nn', 'correction': 'skills', 'chars': {5, 6}}, {'teacher_id': 9, 'type': 'Vform', 'correction': 'decrease', 'chars': {13, 14, 15}}), ('type_error', {'teacher_id': 8, 'type': 'Nn', 'correction': 'skills', 'chars': {5, 6}}, {'teacher_id': 9, 'type': 'ArtOrDet', 'correction': 'a', 'chars': {22}}), ('type_error', {'teacher_id': 8, 'type': 'Nn', 'correction': 'skills', 'chars': {5, 6}}, {'teacher_id': 9, 'type': 'Trans', 'correction': 'rather than', 'chars': {24, 25}}), ('type_error', {'teacher_id': 8, 'type': 'Nn', 'correction': 'skills', 'chars': {5, 6}}, {'teacher_id': 9, 'type': 'ArtOrDet', 'correction': 'a', 'chars': {26}}), ('type_error', {'teacher_id': 8, 'type': 'Vform', 'correction': 'have been', 'chars': {8, 7}}, {'teacher_id': 9, 'type': 'ArtOrDet', 'correction': 'a', 'chars': {22}}), ('type_error', {'teacher_id': 8, 'type': 'Vform', 'correction': 'have been', 'chars': {8, 7}}, {'teacher_id': 9, 'type': 'Trans', 'correct

('40', '3', '1')
[('sets_error', {'teacher_id': 8, 'type': 'ArtOrDet', 'correction': 'the', 'chars': {0, 1}}, {'teacher_id': 9, 'type': 'ArtOrDet', 'correction': 'the', 'chars': {0}}), ('type_error', {'teacher_id': 8, 'type': 'ArtOrDet', 'correction': 'the', 'chars': {0, 1}}, {'teacher_id': 9, 'type': 'Wci', 'correction': 'good', 'chars': {6, 7}}), ('type_error', {'teacher_id': 8, 'type': 'ArtOrDet', 'correction': 'the', 'chars': {0, 1}}, {'teacher_id': 9, 'type': 'Vt', 'correction': 'are becoming', 'chars': {9, 10}}), ('type_error', {'teacher_id': 8, 'type': 'ArtOrDet', 'correction': 'the', 'chars': {0, 1}}, {'teacher_id': 9, 'type': 'Wform', 'correction': 'tech savvy', 'chars': {11, 12}}), ('type_error', {'teacher_id': 8, 'type': 'ArtOrDet', 'correction': 'the', 'chars': {0, 1}}, {'teacher_id': 9, 'type': 'Mec', 'correction': 'other', 'chars': {22, 23}}), ('type_error', {'teacher_id': 8, 'type': 'ArtOrDet', 'correction': 'the', 'chars': {0, 1}}, {'teacher_id': 9, 'type': 'Mec', 'corr

('44', '1', '10')
[('type_error', {'teacher_id': 8, 'type': 'Wci', 'correction': '\n', 'chars': {5, 6, 7}}, {'teacher_id': 9, 'type': 'Prep', 'correction': '\n', 'chars': {5, 6}}), ('sets_error', {'teacher_id': 8, 'type': 'Wci', 'correction': '\n', 'chars': {5, 6, 7}}, {'teacher_id': 9, 'type': 'Prep', 'correction': '\n', 'chars': {5, 6}}), ('type_error', {'teacher_id': 8, 'type': 'Wci', 'correction': '\n', 'chars': {5, 6, 7}}, {'teacher_id': 9, 'type': 'Prep', 'correction': '\n', 'chars': {6, 7}}), ('sets_error', {'teacher_id': 8, 'type': 'Wci', 'correction': '\n', 'chars': {5, 6, 7}}, {'teacher_id': 9, 'type': 'Prep', 'correction': '\n', 'chars': {6, 7}})]
('44', '1', '12')
[('type_error', {'teacher_id': 8, 'type': 'Prep', 'correction': 'and', 'chars': {5, 6}}, {'teacher_id': 9, 'type': 'Rloc-', 'correction': '\n', 'chars': {4, 5, 6}}), ('sets_error', {'teacher_id': 8, 'type': 'Prep', 'correction': 'and', 'chars': {5, 6}}, {'teacher_id': 9, 'type': 'Rloc-', 'correction': '\n', 'chars

('48', '0', '3')
[('type_error', {'teacher_id': 8, 'type': 'Wci', 'correction': 'enable', 'chars': {9, 10, 11}}, {'teacher_id': 9, 'type': 'ArtOrDet', 'correction': '\n', 'chars': {10, 11}}), ('sets_error', {'teacher_id': 8, 'type': 'Wci', 'correction': 'enable', 'chars': {9, 10, 11}}, {'teacher_id': 9, 'type': 'ArtOrDet', 'correction': '\n', 'chars': {10, 11}}), ('type_error', {'teacher_id': 8, 'type': 'Wci', 'correction': 'enable', 'chars': {9, 10, 11}}, {'teacher_id': 9, 'type': 'Rloc-', 'correction': '\n', 'chars': {13, 14}}), ('type_error', {'teacher_id': 8, 'type': 'Wci', 'correction': 'enable', 'chars': {9, 10, 11}}, {'teacher_id': 9, 'type': 'ArtOrDet', 'correction': '\n', 'chars': {16, 17}}), ('type_error', {'teacher_id': 8, 'type': 'Wci', 'correction': 'enable', 'chars': {9, 10, 11}}, {'teacher_id': 9, 'type': 'ArtOrDet', 'correction': '\n', 'chars': {19, 20}}), ('type_error', {'teacher_id': 8, 'type': 'Wci', 'correction': 'enable', 'chars': {9, 10, 11}}, {'teacher_id': 9, 't

('6', '1', '6')
[('type_error', {'teacher_id': 8, 'type': 'SVA', 'correction': 'needs', 'chars': {3, 4}}, {'teacher_id': 9, 'type': 'V0', 'correction': 'be', 'chars': {5}}), ('type_error', {'teacher_id': 8, 'type': 'SVA', 'correction': 'needs', 'chars': {3, 4}}, {'teacher_id': 9, 'type': 'Prep', 'correction': 'into', 'chars': {6, 7}}), ('type_error', {'teacher_id': 8, 'type': 'SVA', 'correction': 'needs', 'chars': {3, 4}}, {'teacher_id': 9, 'type': 'Mec', 'correction': ',', 'chars': {10}}), ('type_error', {'teacher_id': 8, 'type': 'SVA', 'correction': 'needs', 'chars': {3, 4}}, {'teacher_id': 9, 'type': 'Vt', 'correction': 'cause', 'chars': {13, 14}}), ('type_error', {'teacher_id': 8, 'type': 'SVA', 'correction': 'needs', 'chars': {3, 4}}, {'teacher_id': 9, 'type': 'Wci', 'correction': 'chaos', 'chars': {16, 15}}), ('type_error', {'teacher_id': 8, 'type': 'SVA', 'correction': 'needs', 'chars': {3, 4}}, {'teacher_id': 9, 'type': 'Prep', 'correction': 'for', 'chars': {19, 20}}), ('type_e

In [24]:
errors_types=[]
for key in grouped_mistakes:
    for type_,a,b in grouped_mistakes[key]:
        errors_types.append(type_)

In [25]:
len(errors_types)

9684

In [26]:
import collections
collections.Counter(errors_types).most_common()

[('type_error', 8673), ('sets_error', 1011)]

There is 9684 inter annotator conflicts (8673 - annotators marked errors by difernt classes and 1011 - error is the same but spaces are diferent)

# Example:

In [43]:
paragraph_errors=grouped_mistakes[('1', '1', '8')]
paragraph_errors

,teacher_id,type,correction,nid,pid,sid,start_token,end_token,chars
3,8,Mec,diagnosed,1,1,8,3,4,"{2, 3}"
4,8,Prep,\n,1,1,8,4,5,"{3, 4}"
5,8,Nn,diseases,1,1,8,8,9,"{8, 7}"
2395,9,Rloc-,diagnosed,1,1,8,3,5,"{2, 3, 4}"
2396,9,ArtOrDet,a,1,1,8,6,6,{5}
2397,9,Vt,supposed,1,1,8,12,13,"{11, 12}"
